# Выполнение задания 1
### Изменения прописаны капсом

### Импортируем необходимые пакеты

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
from torch import nn
from tqdm.notebook import tqdm

In [2]:
# используем готовый класс от торча для загрузки данных для тренировки
mnist_train = torchvision.datasets.MNIST(
    "./mnist/", train=True, download=True, transform=torchvision.transforms.ToTensor()
)
mnist_val = torchvision.datasets.MNIST(
    "./mnist/", train=False, download=True, transform=torchvision.transforms.ToTensor()
)

# так как это уже унаследованный от Dataset класс, его можно сразу обернуть в даталоадер
train_dataloader = torch.utils.data.DataLoader(
    mnist_train, batch_size=64, shuffle=True, num_workers=1  # 64 ДАЁТ МЕНЕЕ ШУМНЫЕ ГРАДИЕНТЫ
)

val_dataloader = torch.utils.data.DataLoader(
    mnist_val, batch_size=64, shuffle=False, num_workers=1
)

In [4]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),  # ИСПОЛЬЗУЮ СНС (ЛУЧШЕ БЫ КЛАСС, НО ОСТАВЛЮ ЭТО НА СЛЕД. ЗАДАЧИ)
    nn.ReLU(),  # ДОБАВЛЯЕМ НЕЛИНЕЙНОСТЬ
    nn.MaxPool2d(2),  # ПОНИЖАЕМ РАЗМЕРНОСТЬ 28x28 -> 14x14
    # ВТОРОЙ БЛОК
    nn.Conv2d(32, 64, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),  # 14x14 -> 7x7

    nn.Flatten(),  # превращаем картинку 28х28 в вектор размером 784
    nn.Linear(
        64 * 7 * 7, 128  # ДОБАВЛЯЕМ УМНОЖЕНИЕ НА КОЛИЧЕСТВО КАНАЛОВ
    ),  # линейный слой, преобразующий вектор размера 784 в вектор размера 128
    nn.ReLU(),  # нелинейность
    nn.Linear(
        128, 10
    ),  # линейный слой, преобразующий вектор размера 128 в вектор размера 10
)

# создаем оптимизатор, который будет обновлять веса модели
# AdamW ОБЕСПЕЧИВАЕТ СТАБИЛЬНОЕ ОБНОВЛЕНИЕ ПАРАМЕТРОВ
# lr=0.001 СОГЛАСНО ЭМПИРИЧЕСКИМ ПРАВИЛАМ
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
# АДАПТИРУЕТ СКОРОСТЬ ОБУЧЕНИЯ
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=3, factor=0.5
)

### Изменения прописаны капсом

In [7]:
for epoch in range(5):  # МОЖНО СОКРАТИТЬ ДО 3, Т.К. accuracy НЕ МЕНЯТСЯ ПОСЛЕ 3 ПРОГОНА
    for x_train, y_train in tqdm(train_dataloader):  # берем батч из трейн лоадера
        y_pred = model(x_train)  # делаем предсказания
        loss = F.cross_entropy(y_pred, y_train)  # считаем лосс
        loss.backward()  # считаем градиенты обратным проходом
        optimizer.step()  # обновляем параметры сети
        optimizer.zero_grad()  # обнуляем посчитанные градиенты параметров

    model.eval()  # ВКЛЮЧАЕМ РЕЖИМ ОЦЕНКИ
    if epoch % 2 == 0:
        val_loss = []  # сюда будем складывать **средний по бачу** лосс
        val_accuracy = []
        with torch.no_grad():  # на валидации запрещаем фреймворку считать градиенты по параметрам
            for x_val, y_val in tqdm(
                val_dataloader
            ):  # берем батч из валидационного лоадера
                y_pred = model(x_val)  # делаем предсказания
                loss = F.cross_entropy(y_pred, y_val)  # считаем лосс
                val_loss.append(loss.numpy())  # добавляем в массив
                val_accuracy.extend(
                    (torch.argmax(y_pred, dim=-1) == y_val).numpy().tolist()
                )
        scheduler.step(np.mean(val_loss))  # ОБНОВЛЕНИЕ lr
        # печатаем метрики
        print(
            f"Epoch: {epoch}, loss: {np.mean(val_loss)}, accuracy: {np.mean(val_accuracy)}"
        )

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

Epoch: 0, loss: 0.046054091304540634, accuracy: 0.9846


  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

...

Epoch: 2, loss: 0.027880780398845673, accuracy: 0.9898


  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

Epoch: 4, loss: 0.03223597630858421, accuracy: 0.9898


Тем самым accuracy увеличилась с 0.9775 до 0.9898